In [1]:
import matplotlib.pyplot as plt 
import plotly.express as px 
import plotly.graph_objects as go 
import utils.utils as utils 
from utils.BeneficiaryProfile import Beneficiary
import utils.plotting as plotting
import os 
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import copy 

#### 1. Finding Cliff Points 
#### 2. Displaying Cliffs (& Benefits Programs Lost)

---

##### 0. Load Data From Script 


In [2]:
p2_child3_fp = os.path.join('projects', 'p2_child3.yaml')
p2_child3 = Beneficiary.from_yaml(p2_child3_fp)

In [3]:
p2_child3.Profile

{'ruleYear': [2023],
 'Year': [2023],
 'agePerson1': [30],
 'agePerson2': [30],
 'agePerson3': [8],
 'agePerson4': [5],
 'agePerson5': [2],
 'agePerson6': ['NA'],
 'agePerson7': ['NA'],
 'agePerson8': ['NA'],
 'agePerson9': ['NA'],
 'agePerson10': ['NA'],
 'agePerson11': ['NA'],
 'agePerson12': ['NA'],
 'disability1': [0],
 'disability2': [0],
 'disability3': [0],
 'disability4': [0],
 'disability5': [0],
 'disability6': [0],
 'disability7': [0],
 'disability8': [0],
 'disability9': [0],
 'disability10': [0],
 'disability11': [0],
 'disability12': [0],
 'blind1': [0],
 'blind2': [0],
 'blind3': [0],
 'blind4': [0],
 'blind5': [0],
 'blind6': [0],
 'ssdiPIA1': [0],
 'ssdiPIA2': [0],
 'ssdiPIA3': [0],
 'ssdiPIA4': [0],
 'ssdiPIA5': [0],
 'ssdiPIA6': [0],
 'married': [0],
 'prev_ssi': [0],
 'locations': ['New Castle County, DE'],
 'income_start': 27560,
 'income_end': 100000,
 'income_increase_by': 1000,
 'income.investment': [0],
 'income.gift': [0],
 'income.child_support': [0],
 'empl_

In [4]:
## Can Re-run R-Script for demonstration: Takes the yaml file associated with the beneficiary object in the projects folder to set configuration parameters 
# p2_child3.run_applyBenefitsCalculator()

In [5]:
print('Profile Summary:\n')
print("Family")
display(p2_child3.get_family())
print('Locations:', p2_child3.locations)
print("\nBenefits")
print(p2_child3.get_benefits())
print('\nResults:')

df = pd.read_csv(p2_child3.output_path)
display(df.head(),df.shape)
print(df.columns)

Profile Summary:

Family


{'Adult1': {'Age': 30, 'Disability': False, 'Blind': False, 'SSDI_Monthly': 0},
 'Adult2': {'Age': 30, 'Disability': False, 'Blind': False, 'SSDI_Monthly': 0},
 'Child1': {'Age': 8, 'Disability': False},
 'Child2': {'Age': 5, 'Disability': False},
 'Child3': {'Age': 2, 'Disability': False}}

Locations: ['New Castle County, DE']

Benefits
['CHILDCARE', 'HEADSTART', 'CCDF', 'REK', 'HEALTHCARE', 'MEDICAID_ADULT', 'MEDICAID_CHILD', 'CA', 'SECTION8', 'SNAP', 'SLP', 'WIC', 'EITC', 'TAXES', 'CTC', 'CDCTC', 'TANF', 'SSI', 'SSDI']

Results:


,ruleYear,stateFIPS,stateName,stateAbbrev,countyortownName,famsize,numadults,numkids,agePerson1,agePerson2,...,value.ctc.state,value.eitc.fed,value.eitc.state,value.eitc,value.ctc,value.cdctc,value.ssdi,value.ssi,AfterTaxIncome,NetResources
0,2023,10,Delaware,DE,New Castle County,5,2,3,30,30,...,0,5370,1009,6379,6000,371,0,0,23627,-3610.6
1,2023,10,Delaware,DE,New Castle County,5,2,3,30,30,...,0,5159,1032,6191,6000,367,0,0,24397,-3629.6
2,2023,10,Delaware,DE,New Castle County,5,2,3,30,30,...,0,4949,990,5939,6000,363,0,0,25165,-3715.6
3,2023,10,Delaware,DE,New Castle County,5,2,3,30,30,...,0,4738,948,5686,6000,366,0,0,25933,-3795.6
4,2023,10,Delaware,DE,New Castle County,5,2,3,30,30,...,0,4528,906,5434,6000,378,0,0,26701,-3864.6


(73, 59)

Index(['ruleYear', 'stateFIPS', 'stateName', 'stateAbbrev', 'countyortownName',
       'famsize', 'numadults', 'numkids', 'agePerson1', 'agePerson2',
       'agePerson3', 'agePerson4', 'agePerson5', 'agePerson6', 'agePerson7',
       'agePerson8', 'agePerson9', 'agePerson10', 'agePerson11', 'agePerson12',
       'empl_healthcare', 'income', 'assets.cash', 'exp.childcare', 'exp.food',
       'exp.rentormortgage', 'exp.healthcare', 'exp.utilities', 'exp.misc',
       'exp.transportation', 'netexp.childcare', 'netexp.food',
       'netexp.rentormortgage', 'netexp.healthcare', 'netexp.utilities',
       'value.snap', 'value.schoolmeals', 'value.section8', 'value.liheap',
       'value.medicaid.adult', 'value.medicaid.child', 'value.aca',
       'value.employerhealthcare', 'value.CCDF', 'value.HeadStart',
       'value.PreK', 'value.cdctc.fed', 'value.cdctc.state', 'value.ctc.fed',
       'value.ctc.state', 'value.eitc.fed', 'value.eitc.state', 'value.eitc',
       'value.ctc', 'value.cdctc

In [6]:
df_benefits = df.filter(regex='value')

# Only interested in plotting benefits which could be producing a cliff, i.e. if it drops to zero at any point 
df_benefits_filtered = df_benefits.loc[:, ((df_benefits != 0).any(axis=0) & (df_benefits == 0).any(axis=0))] 
df_benefits_filtered

,value.snap,value.section8,value.medicaid.adult,value.medicaid.child,value.aca,value.CCDF,value.eitc.fed,value.eitc.state,value.eitc
0,10182,13963,16070,13629,0,13626.1,5370,1009,6379
1,9913,13675,16070,13629,0,13586.1,5159,1032,6191
2,9643,13387,16070,13629,0,13546.1,4949,990,5939
3,9373,13099,16070,13629,0,13506.1,4738,948,5686
4,9104,12811,16070,13629,0,13466.1,4528,906,5434
...,...,...,...,...,...,...,...,...,...
68,0,0,0,0,16345,0.0,0,0,0
69,0,0,0,0,16172,0.0,0,0,0
70,0,0,0,0,16038,0.0,0,0,0
71,0,0,0,0,15946,0.0,0,0,0


#### 1. Find Cliff Points 

##### Based on an initial view of the plot...

In [7]:
import plotly.express as px 
import plotly.graph_objects as go 

## Color map for the line plot and the legend 

## Plot 

x_var='AfterTaxIncome'
x_var='income'
## -- Base plot, Net Resources -- ## 
fig = px.line(df, x=x_var, y='NetResources', 
              color_discrete_sequence=[plotting.beneficiary_color_palette[1]], 
              title='Net Resources in New Castle County, DE')

fig.update_traces(mode='markers+lines')


## -- Benefits Programs  -- ## 
for n, col in enumerate(df_benefits_filtered): 
    visibility = 'legendonly' if 'eitc' in col or 'section8' in col else True
    fig.add_trace(go.Scatter(x=df[x_var],y=df[col], mode='markers+lines', line=dict(color=plotting.ben_display_map[col][1]), name=col, visible=visibility))


## -- Derivative  -- ## 
x,y = df[x_var], df['NetResources'],
derivative_scale = 10000
fig.add_trace(go.Scatter(x=x,y=np.gradient(y,x) * derivative_scale, mode='markers+lines', line=dict(color="grey"), name=f'1st Derivative (* {derivative_scale})'))
# fig.add_trace(go.Scatter(x=x,y=np.gradient(np.gradient(y, x) * 1000, x) * derivative_scale, mode='markers+lines', line=dict(color="black"), name=f'2nd Derivative (* {derivative_scale})'))


## --- Customizing Hover Text --- ## 
# fig.update_traces()


## -- Add baseline (break-even) -- ## 
fig.add_shape(type="line",
              x0=df[x_var].min(),
              y0=0,
              x1=df[x_var].max(),
              y1=0,
              line=dict(color="red",width=4,dash="dash"),
              )

## --- Adding Legend(s) --- ## 

# legend_text = f'<span style="{color_palette[2]}"><b>Family</b></span> <i>(Age, Disabled, Blind, Monthly SSDI)<br></i>'
legend_text = f"<span style='color:{plotting.beneficiary_color_palette[1]};'><b>Family:</b></span> <i>(Age, Disabled, Blind, Monthly SSDI)<br></i>"

legend_info = p2_child3.get_family()

for fam_member, params in legend_info.items():
    legend_text = legend_text + f"<b>{fam_member}</b>: " 
    for v in params.values(): 
        legend_text += f"{v}, "
    legend_text = legend_text.rstrip(', ') + "<br>"
    
## Add Benefits List
legend_text += "<i>" 
ben_list = p2_child3.get_benefits()
for i in range(len(ben_list)):
    if i % 3 == 0:
        legend_text += "<br>"
    legend_text += ben_list[i] + ", " 
legend_text = legend_text.rstrip(",  ") + "</i>"


fig.add_annotation(
    text=legend_text,
    align='left',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=1.4,
    y=0,

    bordercolor='black',
    borderwidth=1
)

# Update layout to adjust the legend position
fig.update_layout(
    xaxis=dict(title=x_var),
    yaxis=dict(title='NetResources'),
    margin=dict(l=60, r=400, t=60, b=80), # margins for annotation
    height=600,
    width=1200

)


fig.show()


* Cliff points (in the green trendline) coincide with points where a certain benefits program hits zero.

* For this plot,  I've filtered the benefits programs in the legend to those which contain a zero.<br> - Since EITC never has a cliff (it's a percentage up to a maximum), I left it transparent. <br> - Section8 doesn't have a cliff here but I'm not confident that's always the case 

<!-- Could use local minima of the derivative (where negative), but that could be too broad 
* E.g. note derivative at $42,452  
1. Look at distribution of the (second?) derivative and find outliers?  -->
##### **Two-part rule-based approach**: 

Find points where 
1. Derivative is negative and a local minimum 
2. A benefits program just hit zero 



In [8]:
## Find local minima of derivative
derivative = np.gradient(y,x)
derivative_rel_minima = argrelextrema(derivative, np.less)[0]
# drop non-negative local minima (derivative obviously must be negative to be a cliff)
derivative_rel_minima = derivative_rel_minima[derivative_rel_minima > 0] 

## Find zeros of relevant benefits programs 

# Filter to relevant benefits programs 
df_benefits = df.filter(regex='value')
# Benefits program must have some non-zero values and some zero values 
df_benefits_filtered = df_benefits.loc[:, ((df_benefits != 0).any(axis=0) & (df_benefits == 0).any(axis=0))]
# We exclude EITC programs by name 
df_benefits_filtered = df_benefits_filtered[[col for col in df_benefits_filtered.columns
                                             if 'eitc' not in col]]
# Find zeros 
benefits_zeros = []
for col in df_benefits_filtered.columns: 
    z_index = df_benefits_filtered[col].to_list().index(0)
    benefits_zeros.append(z_index)
benefits_zeros.sort()

## Find overlaps 
cliff_valleys = []
for z in benefits_zeros: 
    if any(z in pd.Interval(a-1,a+1) for a in derivative_rel_minima): 
        cliff_valleys.append(z)
cliff_peaks = [v - 1 for v in cliff_valleys]

print('Benefits zeros (points where a benefits program is zero) (idx, income level)')
print(x[benefits_zeros].to_dict())

# print('\nLocal Minima of Derivative (negative only)')
# print(x[derivative_rel_minima])

print('\nCliff Valleys (benefits zeros coinciding with negative local minima of derivative)')
print(x[cliff_valleys])

print('\nCliff Peaks (subtract "valleys" by one to get the "peaks" of the cliffs)')
print(x[cliff_peaks])

Benefits zeros (points where a benefits program is zero) (idx, income level)
{0: 27560, 21: 48560, 38: 65560, 43: 70560, 49: 76560, 61: 88560}

Cliff Valleys (benefits zeros coinciding with negative local minima of derivative)
21    48560
38    65560
43    70560
Name: income, dtype: int64

Cliff Peaks (subtract "valleys" by one to get the "peaks" of the cliffs)
20    47560
37    64560
42    69560
Name: income, dtype: int64


##### 2. Adding the Cliff Peaks to the Plot 

In [24]:
profile = copy.copy(p2_child3)
data = pd.read_csv(profile.output_path)
color=plotting.beneficiary_color_palette[1]
after_tax = False 
title = f'Net Resources and Benefits in {profile.locations}'
legend_text = plotting.create_profile_legend_text(family_data=profile.get_family(), 
                                                  ben_list=profile.get_benefits(), 
                                                  color=plotting.beneficiary_color_palette[1])

def plot_single_profile(df, color, title=None, after_tax=False, plot_derivative=False, hide_benefits=False, legend_text=None): 
    """Plot a single profile along with its cliff-causing benefits programs"""

    x_var = 'income' if not after_tax else 'AfterTaxIncome' 

    ## Base Plot (NetResources)
    fig = px.line(df, x=x_var, y='NetResources', 
                color_discrete_sequence=[color], 
                title=title, 
                )
    fig.update_traces(mode='markers+lines')

    ## Derivative  
    x,y = df[x_var], df['NetResources']
    derivative = np.gradient(y,x)
    derivative_rel_minima = argrelextrema(derivative, np.less)[0]
    derivative_rel_minima = derivative_rel_minima[derivative_rel_minima > 0] # non-negative

    if plot_derivative: 
        derivative_scale = 10000
        fig.add_trace(go.Scatter(x=x,y=derivative * derivative_scale, 
                                 mode='markers+lines', line=dict(color="grey"), 
                                 name=f'1st Derivative (* {derivative_scale})'))

    ## Benefits Programs 
    df_benefits = utils.filter_benefits(df)
    for col in df_benefits.columns: 
        visibility = 'legendonly' if (hide_benefits or 'eitc' in col or 'section8' in col) else True # These shouldn't have cliffs 
        fig.add_trace(go.Scatter(x=df[x_var],y=df[col],
                                mode='markers+lines', 
                                line=dict(color=plotting.ben_display_map[col][1]),
                                name=plotting.ben_display_map[col][0], visible=visibility))

    ##  Find Cliff Points & Annotate 
    df_benefits = df_benefits[[col for col in df_benefits.columns if 'eitc' not in col]] # drop eitc 
    zeros = utils.find_zeros(df_benefits)
    cliffs = [{"benefit":list(z.keys())[0],
            "valley":list(z.values())[0], 
            "peak":list(z.values())[0] - 1}
        for z in zeros 
        if any(list(z.values())[0] in pd.Interval(e-1,e+1, closed='both')
            for e in derivative_rel_minima)]


    # Add annotation of the program which is lost 
    for cliff in cliffs:
        print(cliff)
        try:
            income_level = x[cliff['peak']]
            income_level_str = utils.format_int_dollars(income_level)
            resource_level = y[cliff['peak']]     
            
            fig.add_annotation(
                x=income_level,
                y=resource_level,
                xref="x",
                yref="y",
                text=f"{plotting.ben_display_map[cliff['benefit']][0]}<br>{income_level_str}",
                showarrow=True,
                font=dict(
                    # family="Courier New, monospace",
                    size=12,
                    color="#ffffff"
                    ),
                align="center",
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#636363",
                ax=-40,
                ay= 40,
                bordercolor="#c7c7c7",
                borderwidth=2,
                borderpad=4,
                bgcolor=plotting.ben_display_map[cliff['benefit']][1],
                opacity=0.8
                )
        except Exception as e: 
            print(cliffs)
            print(cliff['peak'])

    ## --- Customizing Hover Text --- ## 


    ## -- Add baseline (break-even) -- ## 
    fig.add_shape(type="line",
                x0=df[x_var].min(),
                y0=0,
                x1=df[x_var].max(),
                y1=0,
                line=dict(color="red",width=4,dash="dash"),
                )

    ## --- Adding Legend(s) --- ## 
    if legend_text is not None: 
        fig.add_annotation(
            text=legend_text,
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            x=1.4,
            y=0,
            bordercolor='black',
            borderwidth=1
        )

    # Update layout to adjust the legend position
    fig.update_layout(
        xaxis=dict(title=x_var.title()),
        yaxis=dict(title=''),
        margin=dict(l=60, r=400, t=60, b=80), # margins for annotation
        height=600,
        width=1200

    )
    
    ## --- Customize Axis --- ## 
    fig.update_xaxes(labelalias={f"{n}k":f"${n},000 <br>(${(n * 1000) / (52 * 40):.2f}/h)" for n in range(30, 110, 10)})
    # fig.update_yaxes(labelalias={f"{n}k":f"${n},000" for n in range(-15, 25, 5)})


    ## --- Show Plot --- ## 

    fig.show()

    # for cliff in cliffs: 
    #     print(cliff)

plot_single_profile(data, 
                    color=color, 
                    after_tax=after_tax, 
                    title=title,
                    legend_text=legend_text)

{'benefit': 'value.medicaid.adult', 'valley': 21, 'peak': 20}
{'benefit': 'value.CCDF', 'valley': 38, 'peak': 37}
{'benefit': 'value.snap', 'valley': 43, 'peak': 42}


##### Validating for more beneficiary profiles 

In [15]:
import re 
profile_paths = [os.path.join('projects',p) for p in os.listdir('projects') if re.search('p[0-9]_child[0-9]{1}.yaml',p)]
profile_paths.sort()

def create_profile_title(project_name): 
    parent, child = project_name.split('_')
    n_parent, n_child = parent[-1], child.rstrip('.yaml')[-1]
    parent_text = f'{n_parent} Adult'
    if int(n_parent) != 1: 
        parent_text += "s"
    child_text = f"{n_child} Child"
    if int(n_child) != 1: 
        child_text += "ren"

    return parent_text + ", " + child_text

title_map = {os.path.basename(fp).rstrip('.yaml'):create_profile_title(os.path.basename(fp)) for fp in profile_paths}
title_map

{'p1_child0': '1 Adult, 0 Children',
 'p1_child1': '1 Adult, 1 Child',
 'p1_child2': '1 Adult, 2 Children',
 'p1_child3': '1 Adult, 3 Children',
 'p2_child0': '2 Adults, 0 Children',
 'p2_child1': '2 Adults, 1 Child',
 'p2_child2': '2 Adults, 2 Children',
 'p2_child3': '2 Adults, 3 Children'}


Bug sometimes with ACA:
* ACA can be zero at a point where Medicaid is zero and the derivative is a negative local minimum
    * ACA is then counted among the cliff-causing benefit programs 
    * I find the first zero of ACA for the cliff valley
        * It's zero at the first index (inactive if Medicaid is active)
        * Subtract one to get peak, that's -1 
        * Index error

If I error handle by skipping ACA (as I do below) then I might miss actual cliffs with ACA
    
     

In [16]:
for n,fp in enumerate(profile_paths):

    if n not in (2,5):
        continue

    p = Beneficiary.from_yaml(fp)

    plot_derivative = False if n != 2 else True 
    


    df = pd.read_csv(p.output_path)
    color = plotting.beneficiary_color_palette[n % len(plotting.beneficiary_color_palette)] # I only have four colors 

    legend_text = plotting.create_profile_legend_text(family_data=p.get_family(), 
                                                      ben_list=p.get_benefits(), 
                                                      color=color)

    plot_single_profile(df, 
                    color,
                    after_tax=False,
                    hide_benefits=True,
                    plot_derivative=plot_derivative,
                    title=f'Net Resources and Benefits in {p.locations[0]} ({title_map[p.project_name]})', 
                    legend_text=legend_text)

    

##### **Fix second step in method**
Find points where 
1. Derivative is negative and a local minimum 
2. A benefits program *just hit* zero
    * Require that previous point is non-zero

In [17]:
## Fixing algorithm to find zeros 
def find_zero_intercepts(ls:list):
    """Get points of curve (iterable) which first intersect with x-axis (0) (i.e. previous value must be non-zero)""" 

    zero_indices = []
    on_zero = True  # treat first element in list as if we're already on zero (there's no previous point)
    for n in range(len(ls)):
        if ls[n] == 0:
            if not on_zero: # i.e. we found a zero and we're not already on the axis 
                zero_indices.append(n)
                on_zero = True
            else: 
                continue
        else: 
            on_zero = False

    return zero_indices

## Example data 
len_ls = 20
my_ls = [np.random.randint(low=0, high=5) for i in range(len_ls)]
zero_inserts = [np.random.randint(low=0, high=n) for n in range(1,len_ls)]
for n in zero_inserts: 
    my_ls.insert(n,0)

print(my_ls)
print(find_zero_intercepts(my_ls), [my_ls[x-1] for x in find_zero_intercepts(my_ls)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 3, 0, 4, 2, 1, 1, 2, 0, 1, 1, 2, 4, 2, 2, 4, 0, 0, 0]
[22, 28, 36] [3, 2, 4]


In [ ]:
## Create function to calculate derivative rel minima 
## Replace old function to find benefits zeros and cliffs using new find zeros function (consolidate into one function)

In [31]:
def plot_single_profile(df, color, title=None, after_tax=False, plot_derivative=False, hide_benefits=False, legend_text=None): 
    """Plot a single profile along with its cliff-causing benefits programs"""

    x_var = 'income' if not after_tax else 'AfterTaxIncome' 

    ## Base Plot (NetResources)
    fig = px.line(df, x=x_var, y='NetResources', 
                color_discrete_sequence=[color], 
                title=title, 
                )
    fig.update_traces(mode='markers+lines')

    ## Derivative  
    x,y = df[x_var], df['NetResources']
    derivative = np.gradient(y,x)
    derivative_rel_minima = argrelextrema(derivative, np.less)[0]
    derivative_rel_minima = derivative_rel_minima[derivative_rel_minima > 0] # non-negative

    if plot_derivative: 
        derivative_scale = 10000
        fig.add_trace(go.Scatter(x=x,y=derivative * derivative_scale, 
                                 mode='markers+lines', line=dict(color="grey"), 
                                 name=f'1st Derivative (* {derivative_scale})'))

    ## Benefits Programs 
    df_benefits = utils.filter_benefits(df, include_eitc=True)
    for col in df_benefits.columns: 
        visibility = 'legendonly' if (hide_benefits or 'eitc' in col or 'section8' in col) else True # These shouldn't have cliffs 
        fig.add_trace(go.Scatter(x=df[x_var],y=df[col],
                                mode='markers+lines', 
                                line=dict(color=plotting.ben_display_map[col][1]),
                                name=plotting.ben_display_map[col][0], visible=visibility))

    ##  Find Cliff Points & Annotate 
    df_benefits = df_benefits[[col for col in df_benefits.columns if 'eitc' not in col]] # drop eitc 
    cliffs = utils.find_benefits_cliffs(df_benefits, derivative_rel_minima, mode='peak')

    # [{'benefit':k,'valley':v+1,'peak':v} for k,v in cliffs.items()]

    # Add annotation of the program which is lost 
    for ben_col, cliff_indices  in cliffs.items():
        for cliff_idx in cliff_indices: 
            income_level = x[cliff_idx]
            income_level_str = utils.format_int_dollars(income_level)
            resource_level = y[cliff_idx]     
            print(income_level)
            print(resource_level)
            
            fig.add_annotation(
                x=income_level,
                y=resource_level,
                xref="x",
                yref="y",
                text=f"{plotting.ben_display_map[ben_col][0]}<br>{income_level_str}",
                showarrow=True,
                font=dict(
                    # family="Courier New, monospace",
                    size=12,
                    color="#ffffff"
                    ),
                align="center",
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#636363",
                ax=-40,
                ay= 40,
                bordercolor="#c7c7c7",
                borderwidth=2,
                borderpad=4,
                bgcolor=plotting.ben_display_map[ben_col][1],
                opacity=0.8
                )

    ## --- Customizing Hover Text --- ## 


    ## -- Add baseline (break-even) -- ## 
    fig.add_shape(type="line",
                x0=df[x_var].min(),
                y0=0,
                x1=df[x_var].max(),
                y1=0,
                line=dict(color="red",width=4,dash="dash"),
                )

    ## --- Adding Legend(s) --- ## 
    if legend_text is not None: 
        fig.add_annotation(
            text=legend_text,
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            x=1.4,
            y=0,
            bordercolor='black',
            borderwidth=1
        )

    # Update layout to adjust the legend position
    fig.update_layout(
        xaxis=dict(title=x_var.title()),
        yaxis=dict(title=''),
        margin=dict(l=60, r=400, t=60, b=80), # margins for annotation
        height=600,
        width=1200

    )
    
    ## --- Customize Axis --- ## 
    fig.update_xaxes(labelalias={f"{n}k":f"${n},000 <br>(${(n * 1000) / (52 * 40):.2f}/h)" for n in range(30, 110, 10)})
    # fig.update_yaxes(labelalias={f"{n}k":f"${n},000" for n in range(-15, 25, 5)})


    ## --- Show Plot --- ## 

    fig.show()

    # for cliff in cliffs: 
    #     print(cliff)

plot_single_profile(df, 
                    color=color, 
                    after_tax=after_tax, 
                    title=title,
                    legend_text=legend_text)

49560
-9230.0
49560
-9230.0
33560
-5950.39999999999
91560
9448.60000000001
45560
-9158.39999999999


In [28]:
cliffs = utils.find_benefits_cliffs(df_benefits, derivative_rel_minima, mode='peak')

In [29]:
cliffs

{'value.snap': [42], 'value.medicaid.adult': [20], 'value.CCDF': [37]}

In [19]:
plot_single_profile(df, 
                color,
                after_tax=False,
                hide_benefits=True,
                plot_derivative=plot_derivative,
                title=f'Net Resources and Benefits in {p.locations[0]} ({title_map[p.project_name]})', 
                legend_text=legend_text)

[{'benefit': 'value.aca', 'valley': 0, 'peak': -1}, {'benefit': 'value.medicaid.adult', 'valley': 7, 'peak': 6}, {'benefit': 'value.CCDF', 'valley': 19, 'peak': 18}, {'benefit': 'value.snap', 'valley': 23, 'peak': 22}, {'benefit': 'value.schoolmeals', 'valley': 23, 'peak': 22}]
-1
